In [18]:
from square.client import Client
import os
from dotenv import load_dotenv
import pandas as pd
load_dotenv()

True

In [19]:
client = Client(
    access_token=os.environ['SQUARE_ACCESS_TOKEN'],
    environment='sandbox')

orders_api = client.orders

### Search orders

In [20]:
# Search orders
body = {
    'location_ids': ['LV2XZS82XY1QP'],
    'query': {
        'filter': {
        }
    },
    'limit': 100,
    'return_entries': True
}

result = orders_api.search_orders(body)

if result.is_success():
    print(result.body)
elif result.is_error():
    print(result.errors)


{'order_entries': [{'order_id': '5svy0EhkF9zMrNOu65PHysIT1CgZY', 'version': 4, 'location_id': 'LV2XZS82XY1QP'}, {'order_id': '35gVxVXkWdWou4DRJUXU6bffLGDZY', 'version': 2, 'location_id': 'LV2XZS82XY1QP'}, {'order_id': 'VUS3l0ra1oLrs1G9UcGKctmZbLNZY', 'version': 1, 'location_id': 'LV2XZS82XY1QP'}, {'order_id': 'FU6BBIb5lgNN4ezJo8YYVGBor0CZY', 'version': 1, 'location_id': 'LV2XZS82XY1QP'}, {'order_id': 'TtZ4vOJa0tKIRfUTDOPIVc9y8kSZY', 'version': 1, 'location_id': 'LV2XZS82XY1QP'}, {'order_id': 'dk45czlbXWqudbAMaZ6MUQWaXMKZY', 'version': 1, 'location_id': 'LV2XZS82XY1QP'}, {'order_id': '9M4fYq5tBgDKgDsWNPo7aTaUZOVZY', 'version': 1, 'location_id': 'LV2XZS82XY1QP'}, {'order_id': 'LdHKfiwB6ulvjs8O9TPyMGxiKwLZY', 'version': 1, 'location_id': 'LV2XZS82XY1QP'}, {'order_id': 'pwBwuzR6a1aogOoq7lEXjCQU07cZY', 'version': 1, 'location_id': 'LV2XZS82XY1QP'}, {'order_id': 'tSPtZTkLngcNATT7s4IBWuJD00HZY', 'version': 1, 'location_id': 'LV2XZS82XY1QP'}, {'order_id': 'tIaVbIglQ6cQQiEhJvNcO4YTnM8YY', 'vers

### Get list of order id's

In [21]:
orders_dict = result.body
order_entries_list = orders_dict["order_entries"]

print(len(order_entries_list))
order_id_list = []

for x in range(0, len(order_entries_list)):
    order_id = order_entries_list[x]["order_id"]
    order_id_list.append(order_id)

order_id_list


53


['5svy0EhkF9zMrNOu65PHysIT1CgZY',
 '35gVxVXkWdWou4DRJUXU6bffLGDZY',
 'VUS3l0ra1oLrs1G9UcGKctmZbLNZY',
 'FU6BBIb5lgNN4ezJo8YYVGBor0CZY',
 'TtZ4vOJa0tKIRfUTDOPIVc9y8kSZY',
 'dk45czlbXWqudbAMaZ6MUQWaXMKZY',
 '9M4fYq5tBgDKgDsWNPo7aTaUZOVZY',
 'LdHKfiwB6ulvjs8O9TPyMGxiKwLZY',
 'pwBwuzR6a1aogOoq7lEXjCQU07cZY',
 'tSPtZTkLngcNATT7s4IBWuJD00HZY',
 'tIaVbIglQ6cQQiEhJvNcO4YTnM8YY',
 'Tnu0ns4hQwuqDnv2rCxKgCCrc0IZY',
 'nFujbtaz2XasF4wSujYEFezO9SYZY',
 'VMQ0lBFaVsLhyeE5faUBt630qaXZY',
 'FE2H9jUaHPl1GjG4wfSD7duX9yXZY',
 'FWl2BWkhUhYLQ5jjr0XCAQ0jcZ6YY',
 'TZZXGvPwxKcoEp63C9jkdmnLrcCZY',
 'vZKxzKlebecfC6VwStCcal94E7NZY',
 'n3piYo4zqKg87GhQCzmvPyT2SJKZY',
 'bxdeVgtAynj30jMlUkXof0bkTa7YY',
 'X1MEPZf5RPFdpNchrqX8ffrwdjJZY',
 'ZQ5qGrKWeYNh5kQdmzQoe6QIPDIZY',
 '9eF5skqpFGQXpwU8BXiyEmIrt0FZY',
 'hOYxTQsjjQk0usWYgstnTShfCXIZY',
 '1U2TmHtyhS7ofSBNrDIsF7iSzVQZY',
 'Ze6b2Ydk5EbGoHcfO94la5Xj6RIZY',
 'DDCz4bd18LJ9uAwim3T7hxUqXPYZY',
 '77QR7dfavLyvFlwVAToK2oDKZMHZY',
 '5Ga5WZWmjKhlp13dMWliUMprdGLZY',
 'fzrzcEnH21zt

### Batch retrieve orders

In [22]:
batch_orders = client.orders.batch_retrieve_orders(
  body = {
    "order_ids": order_id_list
  }
)

if batch_orders.is_success():
  print(batch_orders.body)
elif batch_orders.is_error():
  print(batch_orders.errors)

{'orders': [{'id': '5svy0EhkF9zMrNOu65PHysIT1CgZY', 'location_id': 'LV2XZS82XY1QP', 'line_items': [{'uid': 'vM0CBSAwJG9zRaeBoOZxbD', 'catalog_object_id': 'QXCVMRPLR46NRMAN326LPQVY', 'catalog_version': 1694394690403, 'quantity': '1', 'name': 'Croissant', 'variation_name': 'Regular', 'base_price_money': {'amount': 375, 'currency': 'USD'}, 'gross_sales_money': {'amount': 375, 'currency': 'USD'}, 'total_tax_money': {'amount': 0, 'currency': 'USD'}, 'total_discount_money': {'amount': 0, 'currency': 'USD'}, 'total_money': {'amount': 375, 'currency': 'USD'}, 'variation_total_price_money': {'amount': 375, 'currency': 'USD'}, 'item_type': 'ITEM', 'total_service_charge_money': {'amount': 0, 'currency': 'USD'}}], 'created_at': '2023-09-11T01:33:06.892Z', 'updated_at': '2023-09-11T01:33:07.784Z', 'state': 'OPEN', 'version': 4, 'total_tax_money': {'amount': 0, 'currency': 'USD'}, 'total_discount_money': {'amount': 0, 'currency': 'USD'}, 'total_tip_money': {'amount': 0, 'currency': 'USD'}, 'total_mo

### List of orders

In [23]:
batch_orders_dict = batch_orders.body
batch_orders_list = batch_orders_dict['orders']

In [24]:
batch_orders_list

[{'id': '5svy0EhkF9zMrNOu65PHysIT1CgZY',
  'location_id': 'LV2XZS82XY1QP',
  'line_items': [{'uid': 'vM0CBSAwJG9zRaeBoOZxbD',
    'catalog_object_id': 'QXCVMRPLR46NRMAN326LPQVY',
    'catalog_version': 1694394690403,
    'quantity': '1',
    'name': 'Croissant',
    'variation_name': 'Regular',
    'base_price_money': {'amount': 375, 'currency': 'USD'},
    'gross_sales_money': {'amount': 375, 'currency': 'USD'},
    'total_tax_money': {'amount': 0, 'currency': 'USD'},
    'total_discount_money': {'amount': 0, 'currency': 'USD'},
    'total_money': {'amount': 375, 'currency': 'USD'},
    'variation_total_price_money': {'amount': 375, 'currency': 'USD'},
    'item_type': 'ITEM',
    'total_service_charge_money': {'amount': 0, 'currency': 'USD'}}],
  'created_at': '2023-09-11T01:33:06.892Z',
  'updated_at': '2023-09-11T01:33:07.784Z',
  'state': 'OPEN',
  'version': 4,
  'total_tax_money': {'amount': 0, 'currency': 'USD'},
  'total_discount_money': {'amount': 0, 'currency': 'USD'},
  'to

### Orders dataframe

In [25]:
# first_order = batch_orders_list[0]
order_list = []

for order in batch_orders_list:
    id = order["id"]
    location_id = order["location_id"]
    created_at = order["created_at"]
    updated_at = order["updated_at"]
    state = order["state"]
    version = order["version"]
    net_amounts = order["net_amounts"]
    customer_id = order.get("customer_id")
    line_items = order["line_items"]

    cat_obj_id_list = []
    for item in line_items:
        # print(item)
        cat_obj_id_list.append(item["catalog_object_id"])

    order_list.append((id, location_id, created_at, updated_at, state, version, net_amounts, 
                       customer_id, cat_obj_id_list))

# create DataFrame using data
order_df = pd.DataFrame(order_list, columns =["id", "location_id", "created_at", "updated_at", 
                                              "state", "version", "net_amounts", "customer_id", "cat_obj_id"])

order_df = order_df.explode("cat_obj_id")
order_df.to_csv("./graph_data/orders.csv", index=False)
order_df.head()

,id,location_id,created_at,updated_at,state,version,net_amounts,customer_id,cat_obj_id
0,5svy0EhkF9zMrNOu65PHysIT1CgZY,LV2XZS82XY1QP,2023-09-11T01:33:06.892Z,2023-09-11T01:33:07.784Z,OPEN,4,"{'total_money': {'amount': 375, 'currency': 'U...",None,QXCVMRPLR46NRMAN326LPQVY
1,35gVxVXkWdWou4DRJUXU6bffLGDZY,LV2XZS82XY1QP,2023-09-11T01:24:12.881Z,2023-09-11T01:33:05.996Z,DRAFT,2,"{'total_money': {'amount': 375, 'currency': 'U...",None,QXCVMRPLR46NRMAN326LPQVY
2,VUS3l0ra1oLrs1G9UcGKctmZbLNZY,LV2XZS82XY1QP,2023-09-11T00:27:53.013Z,2023-09-11T00:27:53.013Z,DRAFT,1,"{'total_money': {'amount': 375, 'currency': 'U...",None,QXCVMRPLR46NRMAN326LPQVY
3,FU6BBIb5lgNN4ezJo8YYVGBor0CZY,LV2XZS82XY1QP,2023-09-09T02:09:50.182Z,2023-09-09T02:09:50.182Z,OPEN,1,"{'total_money': {'amount': 1100, 'currency': '...",VK09BGAVBNB1T5F4530K163W5M,QU2JOEL4HG64KCTGLUSLFUK6
4,TtZ4vOJa0tKIRfUTDOPIVc9y8kSZY,LV2XZS82XY1QP,2023-09-09T02:09:49.475Z,2023-09-09T02:09:49.475Z,OPEN,1,"{'total_money': {'amount': 964, 'currency': 'U...",1P0ZY96849H446231JNDH0V97R,M5XGQR3VGKXQRLU5PCI3KH3C


### Order x Location relationship

In [26]:
order_x_location_df = order_df[["id", "location_id"]]
order_x_location_df = order_x_location_df.drop_duplicates()

order_x_location_df.to_csv("./graph_data/order_x_location.csv", index=False)
order_x_location_df

,id,location_id
0,5svy0EhkF9zMrNOu65PHysIT1CgZY,LV2XZS82XY1QP
1,35gVxVXkWdWou4DRJUXU6bffLGDZY,LV2XZS82XY1QP
2,VUS3l0ra1oLrs1G9UcGKctmZbLNZY,LV2XZS82XY1QP
3,FU6BBIb5lgNN4ezJo8YYVGBor0CZY,LV2XZS82XY1QP
4,TtZ4vOJa0tKIRfUTDOPIVc9y8kSZY,LV2XZS82XY1QP
5,dk45czlbXWqudbAMaZ6MUQWaXMKZY,LV2XZS82XY1QP
6,9M4fYq5tBgDKgDsWNPo7aTaUZOVZY,LV2XZS82XY1QP
7,LdHKfiwB6ulvjs8O9TPyMGxiKwLZY,LV2XZS82XY1QP
8,pwBwuzR6a1aogOoq7lEXjCQU07cZY,LV2XZS82XY1QP
9,tSPtZTkLngcNATT7s4IBWuJD00HZY,LV2XZS82XY1QP


### Order x Customer relationship

In [27]:
order_x_cust_df = order_df[["id", "customer_id"]]
order_x_cust_df = order_x_cust_df.drop_duplicates()

order_x_cust_df.to_csv("./graph_data/order_x_cust.csv", index=False)
order_x_cust_df

,id,customer_id
0,5svy0EhkF9zMrNOu65PHysIT1CgZY,None
1,35gVxVXkWdWou4DRJUXU6bffLGDZY,None
2,VUS3l0ra1oLrs1G9UcGKctmZbLNZY,None
3,FU6BBIb5lgNN4ezJo8YYVGBor0CZY,VK09BGAVBNB1T5F4530K163W5M
4,TtZ4vOJa0tKIRfUTDOPIVc9y8kSZY,1P0ZY96849H446231JNDH0V97R
5,dk45czlbXWqudbAMaZ6MUQWaXMKZY,6WH79TC7KP6X5AYXWQWDR377TW
6,9M4fYq5tBgDKgDsWNPo7aTaUZOVZY,MXSQ3Q1QE8M80HDTVHHJEZRF50
7,LdHKfiwB6ulvjs8O9TPyMGxiKwLZY,T7EM1V4A8J23ZGYNGATK07PNKR
8,pwBwuzR6a1aogOoq7lEXjCQU07cZY,64AS3BG573V3JW759JH9N7K2X8
9,tSPtZTkLngcNATT7s4IBWuJD00HZY,VK09BGAVBNB1T5F4530K163W5M


### Order x Category relationship

In [28]:
order_x_cat_obj_df = order_df[["id", "cat_obj_id"]]
order_x_cat_obj_df = order_x_cat_obj_df.drop_duplicates()

order_x_cat_obj_df.to_csv("./graph_data/order_x_cat_obj.csv", index=False)
order_x_cat_obj_df

,id,cat_obj_id
0,5svy0EhkF9zMrNOu65PHysIT1CgZY,QXCVMRPLR46NRMAN326LPQVY
1,35gVxVXkWdWou4DRJUXU6bffLGDZY,QXCVMRPLR46NRMAN326LPQVY
2,VUS3l0ra1oLrs1G9UcGKctmZbLNZY,QXCVMRPLR46NRMAN326LPQVY
3,FU6BBIb5lgNN4ezJo8YYVGBor0CZY,QU2JOEL4HG64KCTGLUSLFUK6
4,TtZ4vOJa0tKIRfUTDOPIVc9y8kSZY,M5XGQR3VGKXQRLU5PCI3KH3C
5,dk45czlbXWqudbAMaZ6MUQWaXMKZY,G3USA7KKSPWD7ZZ2I2RMQIYL
6,9M4fYq5tBgDKgDsWNPo7aTaUZOVZY,FLXOYUKIRKSDWCSER6VW7EQZ
7,LdHKfiwB6ulvjs8O9TPyMGxiKwLZY,FTJBGLEJPVXCOYXEAXKQSKGO
8,pwBwuzR6a1aogOoq7lEXjCQU07cZY,KDQB3MWNGFSIUKV73MFPB23U
9,tSPtZTkLngcNATT7s4IBWuJD00HZY,JTI4ZU3BYVZH6H6PG5OSZXTK
